In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from collections import defaultdict

def get_instance(df, n_agents, scen_id, solver_name):
    return df[df["n_agents"] == n_agents][df["scen_id"] == scen_id][df["solver_name"] == solver_name]

def get_value(instance, col:str):
    values = instance[col].values
    if len(values) > 1:
        raise ValueError(f"{instance} has multiple values for {col}")
    if len(values) == 0:
        raise ValueError(f"{instance} has no values for {col}")
    
    return values[0]

def get_common_scens(df, all_solvers, n_agents):
    res = []
    for scen_id in range(1, 26):
        try:
            if all([get_value(get_instance(df, n_agents, scen_id, solver), "rate") > 0 for solver in all_solvers]):
                res.append(scen_id)
        except ValueError:
            pass
    
    return res

def total_scen_count(df, n_agents, solver):
    res = 0
    for scen_id in range(1, 26):
        try:
            if get_value(get_instance(df, n_agents, scen_id, solver), "rate") > 0:
                res += 1
        except ValueError:
            pass
    
    return res

def extract_replans_mean(s: str):
    if s == "-":
        return 0
    return float(s.replace("-", ""))
    

def compare(df):
    parameters = ["adr", "rate", "adr_stderr", "total_time", "train_time", "exec_time"]
    res_df = pd.DataFrame(columns=["solver", "agents"] + parameters + ["total_scen_count", "replans_max_size", "replans_mean"])
    
    all_solvers = df['solver_name'].unique()
    for n_agents in df['n_agents'].unique():
        common_scens = get_common_scens(df, all_solvers, n_agents)
        for solver in all_solvers:
            values = defaultdict(lambda: 0)
            replans_max_size = 0
            replans_sum = 0
            for scen_id in common_scens:
                instance = get_instance(df, n_agents, scen_id, solver)
                for param in parameters:
                    values[param] += get_value(instance, param)
                    replans_max_size = max(replans_max_size, get_value(instance, "replans_max_size"))
                    replans_sum += extract_replans_mean(get_value(instance, "replans_mean"))
            
            # Add the new row which is aggregated over all of the common scenarios
            res_new_row = {"solver": solver, "agents": n_agents}
            res_new_row.update({param: values[param] / len(common_scens) for param in values.keys()})
            scen_count = total_scen_count(df, n_agents, solver)
            res_new_row["total_scen_count"] = scen_count
            res_new_row["replans_max_size"] = replans_max_size
            if scen_count > 0:
                res_new_row["replans_mean"] = replans_sum / scen_count
            else:
                res_new_row["replans_mean"] = 0
            res_df = res_df.append(res_new_row, ignore_index=True)
    
    return res_df


def to_str(s) -> str:
#     if type(s) == str and s.startswith("online_window_"):
#         return s[len("online_window_"):][:-2]
    
    if type(s) == str:
        return s
    
    return str(round(s, 1))

def dataframe_to_latex(df):
    print(" & ".join(df.columns), end=" \\\\")
    print("")
    
    for idx, row in df.iterrows():
        print(" & ".join([to_str(val) for val in row.values]), end=" \\\\")
        print("")

In [3]:
IMPORTANT_COLUMNS = ["solver", "agents", "adr", "adr_stderr", "train_time", "exec_time", "replans_mean", "replans_max_size", "total_scen_count", "rate"]

In [204]:
RESULTS_FILE = "vi/room.csv"
MAP_NAME = "room-64-64-16"

df = pd.read_csv(f"{RESULTS_FILE}", index_col=False)
df = compare(df[df["map_name"] == MAP_NAME])
df
# dataframe_to_latex(df[IMPORTANT_COLUMNS])

/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,solver,agents,adr,rate,adr_stderr,total_time,train_time,exec_time,total_scen_count,replans_max_size,replans_mean
0,online_window_dijkstra_2,2,-476.2460,100.0,7.9388,0.2584,0.0708,0.1876,25,2.0,6.760800
1,online_window_vi_2,2,-409.9316,100.0,5.7872,15.9668,15.8084,0.1584,25,2.0,5.481600
2,online_window_dijkstra_2,4,-599.4690,100.0,8.6190,0.3660,0.1330,0.2330,13,2.0,7.781538
3,online_window_vi_2,4,-517.9860,100.0,7.1900,31.4280,31.2300,0.1980,13,2.0,5.404615
4,online_window_dijkstra_2,6,-775.1340,100.0,10.4660,0.7540,0.1960,0.5580,5,3.0,15.408000
5,online_window_vi_2,6,-670.5740,100.0,9.0520,48.6920,48.0060,0.6860,7,3.0,9.171429


In [4]:
RESULTS_FILE = "after_bug/room_ost_random.csv"
MAP_NAME = "room-64-64-16"

df = pd.read_csv(f"{RESULTS_FILE}", index_col=False)
df = compare(df[df["map_name"] == MAP_NAME])
df
# dataframe_to_latex(df[IMPORTANT_COLUMNS])

/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,solver,agents,adr,rate,adr_stderr,total_time,train_time,exec_time,total_scen_count,replans_max_size,replans_mean
0,online_window_dijkstra_2,2,-476.728400,100.0,7.597600,0.192400,0.049600,0.142800,25,2.0,6.914400
1,online_window_vi_2,2,-410.926800,100.0,6.009600,12.366000,12.229200,0.136800,25,2.0,5.971200
2,online_window_dijkstra_2,4,-868.916000,100.0,11.383500,2.178000,0.096000,2.082000,20,3.0,23.205000
3,online_window_vi_2,4,-752.911500,100.0,10.220500,28.364000,24.785000,3.579000,23,3.0,18.824348
4,online_window_dijkstra_2,6,-1123.459231,100.0,13.339231,4.539231,0.143846,4.395385,14,3.0,34.487143
5,online_window_vi_2,6,-961.035385,100.0,10.873846,40.292308,36.101538,4.190769,17,3.0,23.248235
6,online_window_dijkstra_2,8,-1295.267500,100.0,17.442500,3.530000,0.190000,3.340000,5,3.0,42.996000
7,online_window_vi_2,8,-1157.910000,100.0,13.390000,52.360000,47.850000,4.510000,6,3.0,36.540000


In [205]:
RESULTS_FILE = "vi/empty.csv"
MAP_NAME = "empty-48-48"

df = pd.read_csv(f"{RESULTS_FILE}", index_col=False)
df = compare(df[df["map_name"] == MAP_NAME])
df
# dataframe_to_latex(df[IMPORTANT_COLUMNS])

/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,solver,agents,adr,rate,adr_stderr,total_time,train_time,exec_time,total_scen_count,replans_max_size,replans_mean
0,online_window_dijkstra_2,2,-174.097200,100.0,5.154800,0.098400,0.027200,0.071200,25,2.0,2.808000
1,online_window_vi_2,2,-139.323200,100.0,3.372800,4.259600,4.234000,0.025600,25,2.0,0.936000
2,online_window_dijkstra_2,4,-341.217619,100.0,7.896667,0.226190,0.055714,0.170476,22,2.0,6.545455
3,online_window_vi_2,4,-275.793333,100.0,4.800952,9.061429,8.952857,0.108571,22,2.0,4.336364
4,online_window_dijkstra_2,6,-469.540000,100.0,8.876000,0.354000,0.084000,0.270000,8,3.0,5.677500
5,online_window_vi_2,6,-381.006000,100.0,6.016000,14.700000,14.436000,0.264000,11,2.0,4.674545


In [ ]:
RESULTS_FILE = "after_bug/empty_maze.csv"
MAP_NAME = "empty-48-48"

df = pd.read_csv(f"{RESULTS_FILE}", index_col=False)
df = compare(df[df["map_name"] == MAP_NAME])
df
# dataframe_to_latex(df[IMPORTANT_COLUMNS])

/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [206]:
RESULTS_FILE = "vi/maze.csv"
MAP_NAME = "maze-32-32-4"

df = pd.read_csv(f"{RESULTS_FILE}", index_col=False)
df = compare(df[df["map_name"] == MAP_NAME])
df
# dataframe_to_latex(df[IMPORTANT_COLUMNS])

/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,solver,agents,adr,rate,adr_stderr,total_time,train_time,exec_time,total_scen_count,replans_max_size,replans_mean
0,online_window_dijkstra_2,2,-234.756400,100.0,6.110800,0.079200,0.000000,0.079200,25,2.0,5.656800
1,online_window_vi_2,2,-206.437200,100.0,4.850400,2.057200,1.984000,0.073200,25,2.0,5.119200
2,online_window_dijkstra_2,4,-482.201538,100.0,30.892308,0.588462,0.010000,0.578462,14,3.0,29.211429
3,online_window_vi_2,4,-436.767692,100.0,18.296923,4.782308,3.956923,0.825385,13,3.0,33.849231
4,online_window_dijkstra_2,6,-396.435000,100.0,9.840000,0.370000,0.010000,0.360000,2,2.0,26.490000
5,online_window_vi_2,6,-347.065000,100.0,7.520000,5.655000,5.235000,0.420000,2,2.0,30.720000


In [8]:
RESULTS_FILE = "after_bug/empty_maze.csv"
MAP_NAME = "maze-32-32-4"

df = pd.read_csv(f"{RESULTS_FILE}", index_col=False)
df = compare(df[df["map_name"] == MAP_NAME])
df
# dataframe_to_latex(df[IMPORTANT_COLUMNS])

/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,solver,agents,adr,rate,adr_stderr,total_time,train_time,exec_time,total_scen_count,replans_max_size,replans_mean
0,online_window_dijkstra_2,2,-233.447600,100.0,6.197200,0.048000,0.000000,0.048000,25,2.0,4.653600
1,online_window_vi_2,2,-204.286000,100.0,4.716000,1.298400,1.252000,0.046400,25,2.0,4.288800
2,online_window_dijkstra_2,4,-529.507619,100.0,16.739524,2.662381,0.010000,2.652381,21,3.0,35.191429
3,online_window_vi_2,4,-487.070476,100.0,17.571905,5.046667,2.769524,2.277143,21,3.0,34.314286
4,online_window_dijkstra_2,6,-665.370000,100.0,21.718889,10.753333,0.010000,10.743333,11,3.0,50.798176
5,online_window_vi_2,6,-588.804444,100.0,16.782222,14.924444,4.066667,10.857778,11,3.0,54.523631


In [210]:
RESULTS_FILE = "vi/ost_random.csv"
MAP_NAME = "random-64-64-10"

df = pd.read_csv(f"{RESULTS_FILE}", index_col=False)
df = compare(df[df["map_name"] == MAP_NAME])
df
# dataframe_to_latex(df[IMPORTANT_COLUMNS])

/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,solver,agents,adr,rate,adr_stderr,total_time,train_time,exec_time,total_scen_count,replans_max_size,replans_mean
0,online_window_dijkstra_2,2,-232.053200,100.0,4.896000,0.072400,0.050000,0.022400,25,2.0,1.600800
1,online_window_vi_2,2,-197.441600,100.0,3.951600,7.353200,7.337200,0.016000,25,2.0,1.048800
2,online_window_dijkstra_2,4,-446.184762,100.0,7.636190,0.172857,0.100000,0.072857,24,2.0,4.320000
3,online_window_vi_2,4,-382.642857,100.0,5.709048,14.679048,14.604762,0.074286,22,2.0,4.647273
4,online_window_dijkstra_2,6,-721.882727,100.0,10.150909,0.648182,0.150909,0.497273,14,3.0,17.065714
5,online_window_vi_2,6,-608.580000,100.0,8.037273,22.190000,22.020000,0.170000,16,2.0,9.000000
6,online_window_dijkstra_2,8,-846.970000,100.0,10.770000,0.460000,0.190000,0.270000,2,2.0,9.510000
7,online_window_vi_2,8,-741.600000,100.0,10.650000,26.950000,26.680000,0.270000,7,2.0,2.974286
8,online_window_dijkstra_2,10,-960.900000,100.0,14.480000,1.070000,0.250000,0.820000,1,2.0,51.420000
9,online_window_vi_2,10,-729.970000,100.0,8.610000,36.060000,35.680000,0.380000,3,2.0,8.260000


In [5]:
RESULTS_FILE = "after_bug/room_ost_random.csv"
MAP_NAME = "random-64-64-10"

df = pd.read_csv(f"{RESULTS_FILE}", index_col=False)
df = compare(df[df["map_name"] == MAP_NAME])
df
# dataframe_to_latex(df[IMPORTANT_COLUMNS])

/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,solver,agents,adr,rate,adr_stderr,total_time,train_time,exec_time,total_scen_count,replans_max_size,replans_mean
0,online_window_dijkstra_2,2,-232.248800,100.0,5.110800,0.068800,0.049600,0.019200,25,2.0,1.224000
1,online_window_vi_2,2,-199.985600,100.0,3.798800,7.554000,7.538800,0.015200,25,2.0,0.967200
2,online_window_dijkstra_2,4,-484.414000,100.0,7.578800,0.634400,0.096400,0.538000,25,3.0,6.732000
3,online_window_vi_2,4,-415.386800,100.0,5.889200,15.250800,14.776400,0.474400,25,3.0,6.528000
4,online_window_dijkstra_2,6,-761.071818,100.0,10.637727,0.765000,0.146364,0.618636,22,3.0,20.293636
5,online_window_vi_2,6,-647.445000,100.0,7.683636,23.057273,22.233636,0.823636,24,3.0,15.232500
6,online_window_dijkstra_2,8,-1033.742222,100.0,12.694444,2.666667,0.193889,2.472778,21,3.0,29.537143
7,online_window_vi_2,8,-878.362222,100.0,10.070000,32.246111,29.607778,2.638333,22,3.0,26.779091


In [211]:
RESULTS_FILE = "vi/ost_random.csv"
MAP_NAME = "ost003d"

df = pd.read_csv(f"{RESULTS_FILE}", index_col=False)
df = compare(df[df["map_name"] == MAP_NAME])
df = df[df["agents"] < 6]
df
# dataframe_to_latex(df[IMPORTANT_COLUMNS])

/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,solver,agents,adr,rate,adr_stderr,total_time,train_time,exec_time,total_scen_count,replans_max_size,replans_mean
0,online_window_dijkstra_2,2,-484.1212,100.0,8.2348,1.3568,1.3364,0.0204,25,2.0,0.4488
1,online_window_vi_2,2,-381.9088,100.0,5.2680,294.5352,294.5040,0.0312,25,2.0,1.1592
2,online_window_dijkstra_2,4,-950.8176,100.0,11.6932,2.8884,2.8004,0.0880,25,2.0,3.3600
3,online_window_vi_2,4,-754.7224,100.0,7.7688,569.4720,569.3736,0.0984,25,2.0,4.0584


In [212]:
RESULTS_FILE = "vi/ost_longer_timeout.csv"
MAP_NAME = "ost003d"

df = pd.read_csv(f"{RESULTS_FILE}", index_col=False)
df = compare(df[df["map_name"] == MAP_NAME])
# df = df[df["agents"] < 6]
df
# dataframe_to_latex(df[IMPORTANT_COLUMNS])

/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,solver,agents,adr,rate,adr_stderr,total_time,train_time,exec_time,total_scen_count,replans_max_size,replans_mean
0,online_window_dijkstra_2,6,-1487.702381,100.0,14.901429,4.548095,4.406190,0.141905,23,2.0,4.408696
1,online_window_vi_2,6,-1185.027619,100.0,9.590476,905.472857,905.147143,0.325714,23,3.0,6.190435
2,online_window_dijkstra_2,8,-2037.485333,100.0,17.810667,6.269333,5.650667,0.618667,17,3.0,11.145882
3,online_window_vi_2,8,-1605.779333,100.0,11.178667,1215.476667,1215.092000,0.384667,20,2.0,9.213000
4,online_window_dijkstra_2,10,-2531.585714,100.0,18.101429,7.615714,7.130000,0.485714,14,3.0,8.412857
5,online_window_vi_2,10,-2039.904286,100.0,13.141429,1510.221429,1509.624286,0.597143,13,2.0,10.089231
6,online_window_dijkstra_2,12,-3074.000000,100.0,16.475000,9.140000,8.775000,0.365000,7,2.0,3.514286
7,online_window_vi_2,12,-2381.315000,100.0,12.805000,1773.810000,1771.415000,2.395000,4,3.0,15.555000


In [7]:
RESULTS_FILE = "after_bug/room_ost_random.csv"
MAP_NAME = "ost003d"

df = pd.read_csv(f"{RESULTS_FILE}", index_col=False)
df = compare(df[df["map_name"] == MAP_NAME])
df
# dataframe_to_latex(df[IMPORTANT_COLUMNS])

/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,solver,agents,adr,rate,adr_stderr,total_time,train_time,exec_time,total_scen_count,replans_max_size,replans_mean
0,online_window_dijkstra_2,2,-484.2448,100.0,8.3944,1.1824,1.1640,0.0184,25,2.0,0.4488
1,online_window_vi_2,2,-380.5336,100.0,5.2920,253.3216,253.3016,0.0200,25,2.0,0.9912
2,online_window_dijkstra_2,4,-945.7376,100.0,11.7644,2.4380,2.3484,0.0896,25,3.0,3.1680
3,online_window_vi_2,4,-757.0820,100.0,7.6272,502.7444,502.6552,0.0892,25,2.0,4.1496
4,online_window_dijkstra_2,6,-1427.0936,100.0,14.2892,3.7292,3.5248,0.2044,25,3.0,6.5520
5,online_window_vi_2,6,-1144.1108,100.0,9.4904,754.6056,754.1876,0.4180,25,3.0,7.4328
6,online_window_dijkstra_2,8,-1996.2660,100.0,17.7812,5.3724,4.6912,0.6812,25,3.0,15.8064
7,online_window_vi_2,8,-1581.2580,100.0,11.0632,1011.9812,1011.1784,0.8028,25,3.0,13.6896


In [209]:
RESULTS_FILE = "vi/berlin_longer_timeout.csv"
MAP_NAME = "Berlin_1_256"

df = pd.read_csv(f"{RESULTS_FILE}", index_col=False)
df = compare(df[df["map_name"] == MAP_NAME])
df
# dataframe_to_latex(df[IMPORTANT_COLUMNS])

/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,solver,agents,adr,rate,adr_stderr,total_time,train_time,exec_time,total_scen_count,replans_max_size,replans_mean
0,online_window_dijkstra_2,6,-3193.645789,100.0,19.476842,23.327895,22.721053,0.606842,21,2.0,14.297143
1,online_window_vi_2,6,-2594.033158,100.0,12.946316,2478.245789,2477.223684,1.022105,22,2.0,12.016364
2,online_window_dijkstra_2,8,-4103.079375,100.0,21.335625,34.198125,32.736250,1.461875,19,3.0,21.729474
3,online_window_vi_2,8,-3339.064375,100.0,16.143125,3421.351250,3419.461250,1.890000,20,3.0,15.099000


In [ ]:
BASELINE_COLUMNS = ["solver", "agents", "adr", "adr_stderr", "train_time", "total_scen_count"]

In [135]:
from collections import defaultdict

def get_instance(df, n_agents, scen_id, solver_name):
    return df[df["n_agents"] == n_agents][df["scen_id"] == scen_id][df["solver_name"] == solver_name]

def get_value(instance, col:str):
    values = instance[col].values
    if len(values) > 1:
        raise ValueError(f"{instance} has multiple values for {col}")
    if len(values) == 0:
        raise ValueError(f"{instance} has no values for {col}")
    
    res = values[0]
    if col == "train_time" and res==0 and instance["solver_name"].values[0]=="rtdp_dijkstra_rtdp":
        return 180
        
    return res 

def get_common_scens(df, all_solvers, n_agents):
    res = []
    for scen_id in range(1, 26):
        try:
            if all([get_value(get_instance(df, n_agents, scen_id, solver), "rate") > 0 for solver in all_solvers]):
                res.append(scen_id)
        except ValueError:
            pass
    
    return res

def total_scen_count(df, n_agents, solver):
    res = 0
    for scen_id in range(1, 26):
        try:
            if get_value(get_instance(df, n_agents, scen_id, solver), "rate") > 0:
                res += 1
        except ValueError:
            pass
    
    return res


def compare(df):
    parameters = ["adr", "rate", "adr_stderr", "total_time", "train_time", "exec_time"]
    res_df = pd.DataFrame(columns=["solver", "agents"] + parameters + ["total_scen_count", "replans_max_size"])
    
    all_solvers = df['solver_name'].unique()
    for n_agents in df['n_agents'].unique():
        common_scens = get_common_scens(df, all_solvers, n_agents)
        for solver in all_solvers:
            values = defaultdict(lambda: 0)
            replans_max_size = 0
            for scen_id in common_scens:
                instance = get_instance(df, n_agents, scen_id, solver)
                for param in parameters:
                    values[param] += get_value(instance, param)
#                     replans_max_size = max(replans_max_size, get_value(instance, "replans_max_size"))
            
            # Add the new row which is aggregated over all of the common scenarios
            res_new_row = {"solver": solver, "agents": n_agents}
            res_new_row.update({param: values[param] / len(common_scens) for param in values.keys()})
            res_new_row["total_scen_count"] = total_scen_count(df, n_agents, solver)
            res_new_row["replans_max_size"] = replans_max_size
            res_df = res_df.append(res_new_row, ignore_index=True)
    
    return res_df


def to_str(s) -> str:
#     if type(s) == str and s.startswith("online_window_"):
#         return s[len("online_window_"):][:-2]
    
    if type(s) == str:
        return s
    
    return str(round(s, 1))

def dataframe_to_latex(df):
    print(" & ".join(df.columns), end=" \\\\")
    print("")
    
    for idx, row in df.iterrows():
        print(" & ".join([to_str(val) for val in row.values]), end=" \\\\")
        print("")

In [150]:
RESULTS_FILE = "vi/baseline_again.csv"
MAP_NAME = "random-64-64-10"

df = pd.read_csv(f"{RESULTS_FILE}", index_col=False)
df = df[df["n_agents"] == 3][df["solver_name"] != "vi"][df["solver_name"] != "id_rtdp"]
df = compare(df[df["map_name"] == MAP_NAME])
df

# dataframe_to_latex(df[BASELINE_COLUMNS])

/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,solver,agents,adr,rate,adr_stderr,total_time,train_time,exec_time,total_scen_count,replans_max_size
0,online_window_dijkstra_2,3,-338.4272,100.0,6.4080,0.1532,0.1012,0.052,25,0
1,online_window_vi_2,3,-288.9684,100.0,4.8776,15.3796,15.3176,0.062,25,0
2,rtdp_dijkstra_rtdp,3,-296.1052,77.8,5.2116,1.8040,180.0000,1.804,25,0


In [145]:
RESULTS_FILE = "vi/baseline_again.csv"
MAP_NAME = "room-32-32-4"

df = pd.read_csv(f"{RESULTS_FILE}", index_col=False)
df = df[df["n_agents"] == 3][df["solver_name"] != "vi"]
df = compare(df[df["map_name"] == MAP_NAME])
df

# dataframe_to_latex(df[BASELINE_COLUMNS])

/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/levyvonet/.venvs/research/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,solver,agents,adr,rate,adr_stderr,total_time,train_time,exec_time,total_scen_count,replans_max_size
0,online_window_dijkstra_2,3,-130.565,100.0,4.760,0.075,0.010,0.065,8,0
1,online_window_vi_2,3,-124.565,100.0,3.785,1.815,1.745,0.070,6,0
2,id_rtdp,3,-124.485,100.0,4.560,142.740,142.740,0.000,2,0
3,rtdp_dijkstra_rtdp,3,-120.905,62.0,6.310,0.290,180.000,0.290,6,0
